<a href="https://colab.research.google.com/github/kuds/rl-atari-tennis/blob/main/%5BAtari%20Tennis%5D%20Single-Agent%20Reinforcement%20Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Atari Tennis] Single-Agent Reinforcement Learning

In [1]:
!pip install swig

In [2]:
!pip install gymnasium gymnasium[atari] autorom

In [3]:
!AutoROM --accept-license

AutoROM will download the Atari 2600 ROMs.
They will be installed to:
	/usr/local/lib/python3.10/dist-packages/AutoROM/roms

Existing ROMs will be overwritten.


In [4]:
!pip install stable-baselines3

In [5]:
import platform
import torch
import numpy
import stable_baselines3
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack
import gymnasium
from importlib.metadata import version
import time

In [6]:
print(f"Python Version: {platform.python_version()}")
print(f"Torch Version: {version('torch')}")
print(f"Is Cuda Available: {torch.cuda.is_available()}")
print(f"Cuda Version: {torch.version.cuda}")
print(f"Gymnasium Version: {version('gymnasium')}")
print(f"Numpy Version: {version('numpy')}")
print(f"Stable Baselines3 Version: {version('stable_baselines3')}")

Python Version: 3.10.12
Torch Version: 2.4.0+cu121
Is Cuda Available: True
Cuda Version: 12.1
Gymnasium Version: 0.29.1
Numpy Version: 1.26.4
Stable Baselines3 Version: 2.3.2


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
env_str = "ALE/Tennis-v5"
log_dir = "./logs/{}".format(env_str)

In [ ]:
# Create the Atari Tennis environment with appropriate wrappers
env = make_atari_env(env_str, n_envs=1, seed=0)

# Stack frames (usually 4 for Atari games)
env = VecFrameStack(env, n_stack=4)

# Create Evaluation Callback
# eval_freq - can cause learning instability if set to low
eval_callback = stable_baselines3.common.callbacks.EvalCallback(env,
                             best_model_save_path=log_dir,
                             log_path=log_dir,
                             eval_freq=20000,
                             render=False,
                             n_eval_episodes=20)

# Initialize PPO
# ent_coef - encourages exploration of other actins
model = PPO('CnnPolicy',
            env,
            verbose=0,
            ent_coef=0.005)

# Train the model
model.learn(total_timesteps=1000000,
            progress_bar=True,
            callback=eval_callback)

# Save the trained model
model.save('ppo_tennis')

# To visualize the agent's performance, create an unwrapped environment
test_env = gymnasium.make('ALE/Tennis-v5')
test_env = gymnasium.wrappers.AtariPreprocessing(test_env)
test_env = gymnasium.wrappers.FrameStack(test_env, 4)

obs = test_env.reset()
while True:
    # Preprocess observation for the model
    obs_input = numpy.expand_dims(obs, axis=0)
    # Get action from the trained model
    action, _ = model.predict(obs_input, deterministic=True)
    # Step the environment
    obs, reward, done, info = test_env.step(action)
    # Render the environment
    test_env.render()
    if done:
        obs = test_env.reset()

Output()

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/callbacks.py:414: UserWarning: Training and eval env are not of the same type<stable_baselines3.common.vec_env.vec_transpose.VecTransposeImage object at 0x7d84377d6aa0> != <stable_baselines3.common.vec_env.vec_frame_stack.VecFrameStack object at 0x7d836a2fb9d0>
  warnings.warn("Training and eval env are not of the same type" f"{self.training_env} != {self.eval_env}")


Eval num_timesteps=20000, episode_reward=-1.15 +/- 1.68

Episode length: 27000.00 +/- 0.00

New best mean reward!

Eval num_timesteps=40000, episode_reward=-1.75 +/- 3.27

Episode length: 27000.00 +/- 0.00